In [1]:
import warnings
warnings.filterwarnings( 'ignore' )
import gc
import os
import time
import numpy as np
import pandas as pd
import pickle

from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.metrics import f1_score, confusion_matrix
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer

import tensorflow as tf
import keras.backend as K
from keras.models import load_model
from keras.preprocessing import text, sequence


from tqdm import tqdm_notebook
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.util import ngrams

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences


import warnings
warnings.filterwarnings( 'ignore' )

from sklearn.naive_bayes import BernoulliNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
#from thundersvm import SVC
from sklearn.metrics import confusion_matrix, roc_auc_score, accuracy_score


from keras.models import Model
from keras.layers import Input, Embedding
from keras.layers import CuDNNGRU, CuDNNLSTM, Conv1D, Conv2D, Dense, Bidirectional, GRU, LSTM, MaxPool1D
from keras.layers import SpatialDropout1D, Dropout, Concatenate, concatenate, Softmax, Flatten, Reshape
from keras.layers import GlobalMaxPooling1D, GlobalAveragePooling1D, GlobalMaxPooling2D, GlobalAveragePooling2D
from keras.utils import multi_gpu_model
from keras.optimizers import *

Using TensorFlow backend.


from tensorflow.random import set_seed
from numpy.random import seed
import random


seed_value = 0
set_seed(seed_value)
seed(seed_value)
os.environ['PYTHONHASHSEED'] = str(seed_value)
random.seed(seed_value)

In [2]:
import All_RUT_Models
import RUT_Utils

In [3]:
# hyper parameters for this model

penalty = 'l2'
C = 18
solver = 'newton-cg'
class_weight='balanced'

In [4]:
modelname = 'LR'

modelpath = './Models/' + modelname + '/'

if not os.path.exists( modelpath ):
    os.makedirs( modelpath )
if not os.path.exists( './Results/' ):
    os.makedirs( './Results/' )

In [5]:
def hms_string(sec_elapsed):
    h = int(sec_elapsed / (60 * 60))
    m = int((sec_elapsed % (60 * 60)) / 60)
    s = sec_elapsed % 60
    return "{}:{:>02}:{:>05.2f}".format(h, m, s)

In [6]:
def convert_lower_case(data):
    # Convert the input text into lowercase text
    return np.char.lower(str(data))

def remove_stop_words(data):
    # Tokenize the input text and remove stopwords from the corpus
    stop_words = stopwords.words('english')
    lemmatizer = WordNetLemmatizer()
    words = word_tokenize(str(data))
    new_text = ""
    for w in words:
        if w not in stop_words and len(w) > 3:
            new_text = new_text + " " + lemmatizer.lemmatize(w)
    return new_text

def remove_punctuation(data):
    # Remove punctuations defined below from input text
    symbols = "!\"#$%&()*+-./:;<=>?@[\]^_`{|}~\n"
    for i in range(len(symbols)):
        data = np.char.replace(data, symbols[i], ' ')
        data = np.char.replace(data, "  ", " ")
    data = np.char.replace(data, ',', '')
    return data

def remove_apostrophe(data):
    # Remove apostrophe from the input text
    return np.char.replace(data, "'", "")

def preprocess(data):
    # Preprocess the input text
    data = convert_lower_case(data)
    data = remove_punctuation(data) #remove comma seperately
    data = remove_apostrophe(data)
    data = remove_stop_words(data)
    return data

def get_tokens(dataframe, column):
    tokens = []
    for i in tqdm_notebook(dataframe[column][:]):
        _tokens = word_tokenize(str(i))
        tokens.append(_tokens)
        
    return tokens

In [7]:
train_data = pd.read_csv('data\\wiki_debias_train.csv')
train_data = train_data.dropna(axis = 0)
#train_data = train_data.sample(n=100000, random_state=0)
train_data['toxicity'] = train_data['toxicity'].round()

df_test = pd.read_csv('test_data.csv')
df_test = df_test.dropna(axis = 0)
df_test.loc[df_test['Label'] == 'BAD', 'Label'] = 1
df_test.loc[df_test['Label'] == 'NOT_BAD', 'Label'] = 0


train_feature = get_tokens(train_data, 'comment')
train_label = train_data['toxicity']

test_feature = get_tokens(df_test, 'Text')
test_label = df_test['Label']

identity_terms = []
for i in tqdm_notebook(range(len(df_test['Text']))):
    _comment = df_test.loc[i,  'Text'].split(" ")
    if len(_comment) < 3:
        _term = _comment[1]
        identity_terms.append(_term)
identity_terms = list(set(identity_terms))


terms = []
for i in range(len(df_test['Text'])):
    _text = df_test.loc[i, 'Text'].split(' ')
    _term = list(set(_text).intersection(set(identity_terms)))
    if len(_term) > 0:
        terms.append(_term[0])
    else:
        terms.append(np.nan)
        
df_test['Identity_Terms'] = terms

In [8]:
vectorizer = TfidfVectorizer()
vectorizer.fit( train_data['comment'] )

xtrain = vectorizer.transform( train_data['comment'].values )
xtest = vectorizer.transform( df_test[ 'Text' ].values )
ytrain = train_data[ 'toxicity' ]
ytest = df_test[ 'Label' ]


In [ ]:
model = LogisticRegression( penalty=penalty, C=C, solver=solver, class_weight=class_weight )
model.fit( xtrain, ytrain )

In [ ]:
with open( modelpath + modelname +  '.pkl', 'wb' ) as f:
        pickle.dump( model, f )

In [ ]:
with open( modelpath + modelname + '.pkl', 'rb' ) as f:
        model = pickle.load( f )

In [ ]:
pred = model.predict(xtrain)
accuracy_score(ytrain, pred)

In [ ]:
xf_positive = 0
xd_positive = 0
xf_total = 0
xd_total = 0

for i in tqdm_notebook(range(len(train_feature))):
    if(train_data['toxicity'].values[i] == 1 and len(list(set(train_feature[i]).intersection(set(terms)))) > 0):
        xd_positive += 1
        xd_total += 1
    elif(len(list(set(train_feature[i]).intersection(set(terms)))) > 0):
        xd_total += 1
    elif(train_data['toxicity'].values[i] == 1 and len(list(set(train_feature[i]).intersection(set(terms))))==0):
        xf_positive += 1
        xf_total += 1
    elif(len(list(set(train_feature[i]).intersection(set(terms))))== 0):
        xf_total += 1

In [ ]:
pf = xf_positive / xf_total
pd = xd_positive / xd_total
discrimination = pf - pd
discrimination

In [ ]:
pred = model.predict(xtrain)
xf_positive = 0
xd_positive = 0
xf_total = 0
xd_total = 0

for i in tqdm_notebook(range(len(train_feature))):
    if(pred[i].round() == 1 and len(list(set(train_feature[i]).intersection(set(terms)))) > 0):
        xd_positive += 1
        xd_total += 1
    elif(len(list(set(train_feature[i]).intersection(set(terms)))) > 0):
        xd_total += 1
    elif(pred[i].round() == 1 and len(list(set(train_feature[i]).intersection(set(terms))))==0):
        xf_positive += 1
        xf_total += 1
    elif(len(list(set(train_feature[i]).intersection(set(terms))))== 0):
        xf_total += 1

In [ ]:
pf = xf_positive / xf_total
pd = xd_positive / xd_total
discrimination = pf - pd
discrimination

In [ ]:
pred = model.predict(xtest)
df_test['prediction_scores'] = pred
accuracy_score(df_test['Label'].astype(float), pred)

In [ ]:
xf_positive = 0
xd_positive = 0
xf_total = 0
xd_total = 0

for i in tqdm_notebook(range(len(test_feature))):
    if(pred[i].round() == 1 and len(list(set(test_feature[i]).intersection(set(terms)))) > 0):
        xd_positive += 1
        xd_total += 1
    elif(len(list(set(test_feature[i]).intersection(set(terms)))) > 0):
        xd_total += 1
    elif(pred[i].round() == 1 and len(list(set(test_feature[i]).intersection(set(terms))))==0):
        xf_positive += 1
        xf_total += 1
    elif(len(list(set(test_feature[i]).intersection(set(terms))))== 0):
        xf_total += 1
        

In [ ]:
pf = xf_positive / xf_total
pd = xd_positive / xd_total
discrimination = pf - pd
discrimination

In [ ]:
import pandas as pd
def perf_measure(y_actual, y_hat):
    TP = 0
    FP = 0
    TN = 0
    FN = 0

    for i in range(len(y_hat)): 
        if y_actual[i]==y_hat[i]==1:
           TP += 1
        if y_hat[i]==1 and y_actual[i]!=y_hat[i]:
           FP += 1
        if y_actual[i]==y_hat[i]==0:
           TN += 1
        if y_hat[i]==0 and y_actual[i]!=y_hat[i]:
           FN += 1

    return(TN, FP, FN, TP)



total_tn, total_fp, total_fn, total_tp = confusion_matrix(df_test['Label'].astype(float), pred.round()).ravel()
total_fpr = total_fp / (total_fp + total_tn )
total_fnr = total_fn / (total_fn + total_tp)
false_positive = []
false_negative = []
identity_terms = []
for identity_term in set(terms):
    data = df_test[df_test['Identity_Terms'] == identity_term].reset_index()
    y_true, y_pred = data['Label'].astype(int), data['prediction_scores']
    tn, fp, fn, tp = perf_measure(y_true, y_pred.round())
    try:
        fpr = fp / (fp + tn)
        fnr = fn / (fn + tp)
        false_positive.append(fpr)
        false_negative.append(fnr)
        identity_terms.append(identity_term)
    except:
        print("Error in ", identity_term)

    
eval_scores = pd.DataFrame(identity_terms, columns = ['Identity_Titles'])
eval_scores['Identity_Term_False_Positive'] = false_positive
eval_scores['Total_False_Positive'] = total_fpr
eval_scores['Identity_Term_False_Negatives'] = false_negative
eval_scores['Total_False_Negative'] = total_fnr
eval_scores['FPR - FPRt'] = abs(total_fpr - eval_scores['Identity_Term_False_Positive'])
eval_scores['FNR - FNRt'] = abs(total_fnr - eval_scores['Identity_Term_False_Negatives'])
eval_scores

In [ ]:
eval_scores['FPR - FPRt'].sum(), eval_scores['FNR - FNRt'].sum()


In [ ]:
total_auc = roc_auc_score(df_test['Label'].astype(float), pred.round())
terms_auc = []
identity_terms = []
for identity_term in set(terms):
    term_data = df_test[df_test['Identity_Terms'] == identity_term].reset_index()
    data = df_test.sample(n=len(term_data['Text']), random_state=0)
    data = term_data.append(data, ignore_index=True)
    y_true, y_pred = data['Label'].astype(int), data['prediction_scores']

    try:
        term_auc = roc_auc_score(y_true, y_pred.round())
        terms_auc.append(term_auc)
        identity_terms.append(identity_term)
    except:
        print("Error in ",identity_term)


    
eval_scores = pd.DataFrame(identity_terms, columns = ['Identity_Titles'])
eval_scores['AUCt'] = terms_auc
eval_scores['AUC'] = total_auc
eval_scores['AUC - AUCt'] = abs(eval_scores['AUC'] - eval_scores['AUCt'])
eval_scores

In [ ]:
print(eval_scores['AUC - AUCt'].sum())

In [ ]:
pred =  model.predict_proba(xtest)

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
KNN = KNeighborsClassifier(n_neighbors=3, weights = 'distance',n_jobs = -1)
KNN.fit(xtest, ytest.astype('int'))

theta = 0.75
SM_pred = []
for i in tqdm_notebook(range(len(test_feature[:]))):
    p_positive = pred[i][1]
    p_negative = 1 - p_positive
    feature = xtest[i]
    label = df_test.loc[i, 'Label']
    #feature = [np.hstack((feature, label))]
    if max(p_positive, p_negative) < theta:
        prediction = KNN.predict(feature)
        SM_pred.append(prediction.item())
    else:
        SM_pred.append(pred[i][1].round())

In [ ]:
xf_positive = 0
xd_positive = 0
xf_total = 0
xd_total = 0

for i in tqdm_notebook(range(len(test_feature))):
    if(SM_pred[i] == 1 and len(list(set(test_feature[i]).intersection(set(terms)))) > 0):
        xd_positive += 1
        xd_total += 1
    elif(len(list(set(test_feature[i]).intersection(set(terms)))) > 0):
        xd_total += 1
    elif(SM_pred[i] == 1 and len(list(set(test_feature[i]).intersection(set(terms))))== 0):
        xf_positive += 1
        xf_total += 1
    elif(len(list(set(test_feature[i]).intersection(set(terms))))== 0):
        xf_total += 1

In [ ]:
pf = xf_positive / xf_total
pd = xd_positive / xd_total
discrimination = pf - pd
discrimination

In [ ]:
accuracy_score(df_test['Label'].astype(float), SM_pred)

In [ ]:
#test_data['prediction_scores'] = ROC_pred
import pandas as pd
total_tn, total_fp, total_fn, total_tp = confusion_matrix(df_test['Label'].astype(float), SM_pred).ravel()
total_fpr = total_fp / (total_fp + total_tn )
total_fnr = total_fn / (total_fn + total_tp)
false_positive = []
false_negative = []
identity_terms = []
for identity_term in set(terms):
    data = df_test[df_test['Identity_Terms'] == identity_term].reset_index()
    y_true, y_pred = data['Label'].astype(int), data['prediction_scores']
    tn, fp, fn, tp = perf_measure(y_true, y_pred.round())
    try:
        fpr = fp / (fp + tn)
        fnr = fn / (fn + tp)
        false_positive.append(fpr)
        false_negative.append(fnr)
        identity_terms.append(identity_term)
    except:
        print("Error in ", identity_term)

    
eval_scores = pd.DataFrame(identity_terms, columns = ['Identity_Titles'])
eval_scores['Identity_Term_False_Positive'] = false_positive
eval_scores['Total_False_Positive'] = total_fpr
eval_scores['Identity_Term_False_Negatives'] = false_negative
eval_scores['Total_False_Negative'] = total_fnr
eval_scores['FPR - FPRt'] = abs(total_fpr - eval_scores['Identity_Term_False_Positive'])
eval_scores['FNR - FNRt'] = abs(total_fnr - eval_scores['Identity_Term_False_Negatives'])
eval_scores

In [ ]:
eval_scores['FPR - FPRt'].sum(), eval_scores['FNR - FNRt'].sum()

In [ ]:
total_auc = roc_auc_score(df_test['Label'].astype(float), SM_pred)
terms_auc = []
identity_terms = []
for identity_term in set(terms):
    term_data = df_test[df_test['Identity_Terms'] == identity_term].reset_index()
    data = df_test.sample(n=len(term_data['Text']), random_state=0)
    data = term_data.append(data, ignore_index=True)
    y_true, y_pred = data['Label'].astype(int), data['prediction_scores']

    try:
        term_auc = roc_auc_score(y_true, y_pred.round())
        terms_auc.append(term_auc)
        identity_terms.append(identity_term)
    except:
        print("Error in ",identity_term)


    
eval_scores = pd.DataFrame(identity_terms, columns = ['Identity_Titles'])
eval_scores['AUCt'] = terms_auc
eval_scores['AUC'] = total_auc
eval_scores['AUC - AUCt'] = abs(eval_scores['AUC'] - eval_scores['AUCt'])
eval_scores

In [ ]:
print(eval_scores['AUC - AUCt'].sum())